In [ ]:
import os
import io
import pandas as pd
import numpy as np

from dfpp.storage import StorageManager
from dfpp.utils import (
    fix_iso_country_codes, change_iso3_to_system_region_iso3
)
from dfpp.eda import print_essential_data_descriptives

In [ ]:

async with StorageManager() as storage_manager:
    source_file_name = os.path.join(
        storage_manager.sources_path,
        f"{source_cfg['id'].upper()}.{source_cfg['file_format']}",
    )
    
    data = await storage_manager.read_blob(path=source_file_name)

In [ ]:
df = pd.read_csv(io.BytesIO(data), encoding="latin1")

In [ ]:
df.head()

In [ ]:
print_essential_data_descriptives(df)

In [ ]:
source_df = df.copy()

In [ ]:

source_df.rename(columns={"iso3": "Alpha-3 code"}, inplace=True)

rows_to_change_mapping = {
    "ZZA.VHHD": "VHHD",
    "ZZB.HHD": "HHD",
    "ZZC.MHD": "MHD",
    "ZZD.LHD": "LHD",
    "ZZE.AS": "UNDP_AS",
    "ZZF.EAP": "UNDP_EAP",
    "ZZG.ECA": "UNDP_ECA",
    "ZZH.LAC": "UNDP_LAC",
    "ZZI.SA": "UNDP_SA",
    "ZZJ.SSA": "UNDP_SSA",
    "ZZK.WORLD": "WLD",
}
source_df.index = source_df.index.map(
    lambda x: rows_to_change_mapping.get(x, x)
)
source_df.reset_index(inplace=True, drop=True)
source_df = await change_iso3_to_system_region_iso3(source_df, "Alpha-3 code")
source_df.reset_index(inplace=True, drop=True)
await fix_iso_country_codes(df=source_df, col="Alpha-3 code", source_id="HDR")
source_df.rename(
    columns={
            source_cfg["country_name_column"]: "Country",
            source_cfg["country_iso3_column"]: "Alpha-3 code",
    },
    inplace=True,
)

In [ ]:
source_df.head()

In [ ]:

source_df = await change_iso3_to_system_region_iso3(source_df, "Alpha-3 code")
source_df.reset_index(inplace=True)
await fix_iso_country_codes(df=source_df, col="Alpha-3 code", source_id="HDR")
source_df.rename(
    columns={
            source_cfg["country_name_column"]: "Country",
            source_cfg["country_iso3_column"]: "Alpha-3 code",
    },
    inplace=True,
)

In [ ]:
source_df.head()